# Question answering with LLM

In [ ]:
#|default_exp musique.qa

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|hide
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
#|export
import openai

from bellek.logging import get_logger

log = get_logger(__name__)

In [ ]:
# |export

DEFAULT_MODEL = "gpt-3.5-turbo"
DEFAULT_COMPLETION_KWARGS = {"temperature": 0.1}

In [ ]:
# |export

FEW_SHOT_EXAMPLES = [
    {
        "id": "2hop__784447_126070",
        "context": 'Glenhis Hernández (born 7 October 1990 in Havana) is a taekwondo practitioner from Cuba. She was the 2013 World\nChampion in middleweight.\n\nThe current mayor of Havana ("President of the People\'s Power Provincial Assembly") is Marta Hernández Romero, she\nwas elected on March 5, 2011.',
        "question": "Who is the current mayor of Havana?",
        "cte_generation": "Triplets: \nGlenhis Hernández | birth place | Havana\nMarta Hernández Romero | serves as | mayor of Havana\n\nAnswer: Marta Hernández Romero",
        "cot_generation": "Reasoning:\n- The context provides that Glenhis Hernández was born in Havana.\n- The context also specifies that the current mayor of Havana is Marta Hernández Romero, who was elected on March 5, 2011.\n- Since there is no information indicating a change in mayoral leadership since that election, it can be inferred that Marta Hernández Romero remains the mayor.\n",
    },
    {
        "id": "2hop__823584_776926",
        "context": '# Rotst\u00f6ckli\nThe Rotst\u00f6ckli (2,901 m) is a peak of the Urner Alps below the Titlis, on the border between the Swiss cantons of Obwalden and Nidwalden. It is Nidwalden\'s highest point. The summit is split between the municipalities of Engelberg (Obwalden) and Wolfenschiessen (Nidwalden).\n# Uri Alps\nThe Uri Alps (also known as "Urner Alps", ) are a mountain range in Central Switzerland and part of the Western Alps. They extend into the cantons of Obwalden, Valais, Bern, Uri and Nidwalden and are bordered by the Bernese Alps (Grimsel Pass) and the Emmental Alps to the west (the four lakes: Lungerersee, Sarnersee, Wichelsee, and Alpnachersee), the Schwyzer Alps to the north (Lake Lucerne), the Lepontine Alps to the south (the valley of Urseren with Andermatt) and the Glarus Alps to the east (Reuss).',
        "question": "What area contains the region that encompasses Rotst\u00f6ckli?",
        "cte_generation": "Triplets:\nRotst\u00f6ckli | part of | Urner Alps\nUrner Alps | part of | Western Alps\n\nAnswer: Western Alps",
        "cot_generation": "Reasoning:\n1. The Rotstöckli is described as a peak of the Urner Alps.\n2. The Urner Alps are also known as the Uri Alps.\n3. The Uri Alps are a mountain range in Central Switzerland and part of the Western Alps.\n\nAnswer: Western Alps",
    },
]

In [ ]:
DEFAULT_MODEL = "gpt-4-turbo"

TEST_EXAMPLE = {
    "id": "2hop__834974_332063",
    "context": "# N. Monroe Marshall\nNathaniel Monroe Marshall (June 13, 1854 Schuyler Falls, Clinton County, New York \u2013 February 16, 1935 Malone, Franklin County, New York) was an American banker and politician.\n# Perry Township, Clinton County, Indiana\nPerry Township is one of fourteen townships in Clinton County, Indiana. As of the 2010 census, its population was 1,459 and it contained 606 housing units. The township was named for Oliver Hazard Perry, an American naval officer in the War of 1812.",
    "question": "Which region shares border with one where Perry Township is located?",
    "cte_generation": "Triplets:\nNathaniel Monroe Marshall | birth place | Clinton County, New York\nPerry Township | located in | Clinton County, Indiana\n\nAnswer: Franklin County, New York",
}

In [ ]:
#|export

USER_PROMPT = """The context information is provided below.
---------------------
{context}
---------------------
Given the context information and not prior knowledge, answer the question.
{question}
"""

In [ ]:
#|export

EXAMPLE_CONTEXT = """
Glenhis Hernández (born 7 October 1990 in Havana) is a taekwondo practitioner from Cuba. She was the 2013 World
Champion in middleweight.

The current mayor of Havana ("President of the People's Power Provincial Assembly") is Marta Hernández Romero, she
was elected on March 5, 2011.
""".strip()

EXAMPLE_QUESTION = "Who is the current mayor of the city Glenhis Hernández was born?"


### Standard prompt

In [ ]:
#|export

SYSTEM_PROMPT_STANDARD = """
You are an excellent question-answering system known for providing accurate and reliable answers. Your responses should be solely based on the context information given, without drawing on prior knowledge. 

# Output format
Answer: [answer in 2-4 words]
""".strip()

def answer_question_standard(
    context: str,
    question: str,
    model_name: str = DEFAULT_MODEL,
    completion_kwargs: dict | None = None,
    client = None
) -> dict:
    
    if client is None:
        client = openai.Client()
    
    if completion_kwargs is None: 
        completion_kwargs = DEFAULT_COMPLETION_KWARGS
    
    # Prepare the messages
    messages = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT_STANDARD,
        },
        {
            "role": "user",
            "content": USER_PROMPT.format(context=context, question=question),
        },
    ]
    chat_completion = client.chat.completions.create(
            model=model_name,
            messages=messages,
            **completion_kwargs,
        )
    generation = chat_completion.choices[0].message.content
    parts = generation.split("Answer:")
    if len(parts) < 2:
        return dict(answer="", generation=generation)
    answer = parts[1].strip()
    return dict(answer=answer, generation=generation)

In [ ]:
result = answer_question_standard(TEST_EXAMPLE['context'], TEST_EXAMPLE['question'])
print(result['generation'])
print(result['answer'])

Answer: Clinton County, Indiana
Clinton County, Indiana


### Chain-of-thought prompt

In [ ]:
# |export

SYSTEM_PROMPT_COT_FS = """You are an excellent question-answering system known for providing accurate and reliable answers. Your responses should be solely based on the context information given, without drawing on prior knowledge. Always provide clear and logical step-by-step reasoning in your response.

# Output format
Reasoning: [Step-by-step reasoning for the answer.]
Answer: [answer in 2-4 words]
"""

def answer_question_cot_fs(
    context: str,
    question: str,
    examples: list[dict] = FEW_SHOT_EXAMPLES,
    model_name: str = DEFAULT_MODEL,
    completion_kwargs: dict | None = None,
    client=None,
) -> dict:
    if client is None:
        client = openai.Client()

    if completion_kwargs is None:
        completion_kwargs = DEFAULT_COMPLETION_KWARGS

    # Prepare the messages
    messages = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT_COT_FS,
        },
    ]
    for example in examples:
        messages.append(
            {
                "role": "user",
                "content": USER_PROMPT.format(context=example["context"], question=example["question"]),
            }
        )
        messages.append({"role": "assistant", "content": example["cot_generation"]})

    messages.append(
        {
            "role": "user",
            "content": USER_PROMPT.format(context=context, question=question),
        },
    )

    chat_completion = client.chat.completions.create(
        model=model_name,
        messages=messages,
        **completion_kwargs,
    )
    generation = chat_completion.choices[0].message.content
    # Parse the response
    answer = ""
    reasoning = ""
    for line in generation.splitlines():
        if line.startswith("Answer:"):
            answer = line.split("Answer:")[1].strip()
        else:
            reasoning += line.replace("Reasoning:", "") + "\n"
    return dict(reasoning=reasoning.strip(), answer=answer, generation=generation)

In [ ]:
result = answer_question_cot_fs(TEST_EXAMPLE['context'], TEST_EXAMPLE['question'])
print(result['generation'])
print("="*80)
print(result['reasoning'])
print(result['answer'])

Reasoning:
1. Perry Township is located in Clinton County, Indiana.
2. The context does not provide specific information about the neighboring counties or regions directly bordering Clinton County, Indiana.
3. Without additional information on the neighboring regions of Clinton County, Indiana, it is not possible to determine which region shares a border with it based solely on the provided context.

Answer: Information not provided
1. Perry Township is located in Clinton County, Indiana.
2. The context does not provide specific information about the neighboring counties or regions directly bordering Clinton County, Indiana.
3. Without additional information on the neighboring regions of Clinton County, Indiana, it is not possible to determine which region shares a border with it based solely on the provided context.
Information not provided


In [ ]:
def answer_question_cot(
    context: str,
    question: str,
    model_name: str = DEFAULT_MODEL,
    completion_kwargs: dict | None = None,
    client=None,
) -> dict:
    return answer_question_cot_fs(context, question, [], model_name, completion_kwargs, client)

In [ ]:
result = answer_question_cot(TEST_EXAMPLE['context'], TEST_EXAMPLE['question'])
print(result['generation'])
print("="*80)
print(result['reasoning'])
print(result['answer'])

Reasoning:
1. The context information mentions Perry Township as being located in Clinton County, Indiana.
2. The question asks for a region that shares a border with the region where Perry Township is located.
3. Since Perry Township is in Clinton County, Indiana, the answer would be any neighboring county to Clinton County, Indiana.
4. However, the context information does not provide the names of the neighboring counties to Clinton County, Indiana.
5. Therefore, based on the information given, it is not possible to specify which region shares a border with Clinton County, Indiana.

Answer: Cannot specify
1. The context information mentions Perry Township as being located in Clinton County, Indiana.
2. The question asks for a region that shares a border with the region where Perry Township is located.
3. Since Perry Township is in Clinton County, Indiana, the answer would be any neighboring county to Clinton County, Indiana.
4. However, the context information does not provide the na

### Connect-the-Entities prompt

In [ ]:
#|export

SYSTEM_PROMPT_CTE = """
You are an excellent question-answering system known for providing accurate and reliable answers. Your responses should be solely based on the context information given, without drawing on prior knowledge.

Before answering the question, first, you extract relevant entity-relation-entity triplets from the context. Then, you answer the question based on the triplets.

# Output format
Triplets: [A list of entity-relation-entity triplets extracted from the context.]
Answer: [answer in 2-4 words]
""".strip()

def answer_question_cte(
    context: str,
    question: str,
    examples: list[dict] = FEW_SHOT_EXAMPLES,
    model_name: str = DEFAULT_MODEL,
    completion_kwargs: dict | None = None,
    client=None,
) -> dict:
    if client is None:
        client = openai.Client()

    if completion_kwargs is None: 
        completion_kwargs = DEFAULT_COMPLETION_KWARGS
    
    # Prepare the messages
    messages = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT_CTE,
        },
    ]
    for example in examples:
        messages.append(
            {
                "role": "user",
                "content": USER_PROMPT.format(context=example["context"], question=example["question"]),
            }
        )
        messages.append(
            {
                "role": "assistant",
                "content": example["cte_generation"],
            }
        )
    messages.append(
        {
            "role": "user",
            "content": USER_PROMPT.format(context=context, question=question),
        },
    )
    
    # Generate the response
    chat_completion = client.chat.completions.create(
        model=model_name,
        messages=messages,
        **completion_kwargs,
    )
    generation = chat_completion.choices[0].message.content
    
    # Parse the response
    answer = ""
    triplets = []
    for line in generation.splitlines():
        if line.startswith("Answer:"):
            answer = line.split("Answer:")[1].strip()
        elif "|" in line:
            triplets.append(line.strip())
    return dict(triplets=triplets, answer=answer, generation=generation)

In [ ]:
result = answer_question_cte(TEST_EXAMPLE['context'], TEST_EXAMPLE['question'])
print(result['generation'])
print("="*80)
print(result['triplets'])
print(result['answer'])

Triplets:
Perry Township | located in | Clinton County, Indiana

Answer: Clinton County, Indiana
['Perry Township | located in | Clinton County, Indiana']
Clinton County, Indiana


In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()